<a href="https://colab.research.google.com/github/deivid-01/Snakes-Classifier/blob/main/04_model_arquitecture_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model arquitecture and training


### About:
- This notebook focus on train and validate machine learning model.
---
David Andrés Torres Betancour <br/>
Computer Engineering  Student <br/>
University of Antioquia <br/>
davida.torres@udea.edu.co

## Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf

from tensorflow.keras.layers import Flatten, Dense,Dropout
from tensorflow.keras.models  import Model,load_model
from keras.applications.vgg16 import preprocess_input

from keras.applications.vgg16 import preprocess_input
#from keras.applications import vgg16
from tensorflow.keras.applications import VGG16 # Wtf ?
from keras.optimizers import Adam

from google.colab import files
import h5py

%matplotlib inline

## Tools

### Kaggle Tools

In [5]:
class bcolors:
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

def loadCredentials():
  '''
  Description:
    Loads and configure kaggle.json file with credentials.

  Parameters:
      No parameters.

  Returns:
      No return.   
  ''' 
  print(bcolors.BOLD + "Upload kaggle.json file with credentials\n" + bcolors.ENDC)
  print(bcolors.WARNING + "IMPORTANT: Filename must be: kaggle.json\n" + bcolors.ENDC)
  uploaded_file=files.upload() #Upload file
  assert len(uploaded_file)>0,"Fetch canceled: Dataset not loaded"
  assert list(uploaded_file.keys())[0]=='kaggle.json', "Filename must be: kaggle.json"
 


  os.environ['KAGGLE_CONFIG_DIR'] = '/content' #Setup folder
  ! chmod 600 /content/kaggle.json  #Hide kaggle.json


def fetchDatasetFromKaggle(dataset_name, force_fetch=False):
  '''
  Description:
    fetchs dataset  from kaggle.

  Parameters:
      No parameters.

  Returns:
      No return.   
  '''
  if os.path.exists('kaggle.json') == False: 
    loadCredentials() 

  if force_fetch==True:
   process_info=! kaggle datasets download -d deividt/{dataset_name} --force #Download dataset
   if "100%" in list(process_info)[-1]:
       print(bcolors.OKGREEN + "Data from kaggle successfully fetch\n" + bcolors.ENDC)
       print(bcolors.BOLD + "Unzipping data..." + bcolors.ENDC)
       ! unzip \*.zip && rm *.zip
       print(bcolors.OKGREEN + "Data is ready in your local folder!\n" + bcolors.ENDC)
   elif "404 - Not Found" in list(process_info)[0]: 
       print(bcolors.FAIL + "404 - Dataset  Not Found in 'deividt' Account\n" + bcolors.ENDC)
   else:
       assert False,list(process_info)[-1]


  else:
   process_info =  ! kaggle datasets download -d deividt/{dataset_name} 
   if "Skipping" in list(process_info)[0]:
       print(bcolors.WARNING + "Data already exists locally\nIf you want force fetch set force_fetch parameter to True" + bcolors.ENDC)
  
   elif "100%" in list(process_info)[-1]:
       print(bcolors.OKGREEN + "Data from kaggle successfully fetch\n" + bcolors.ENDC)
       print(bcolors.BOLD + "Unzipping data... \n" + bcolors.ENDC)
       ! unzip \*.zip && rm *.zip #Unzip downloaded file and delete zip
       print(bcolors.OKGREEN + "Data is ready in your local folder!\n" + bcolors.ENDC)
   elif "404 - Not Found" in list(process_info)[0]: 
       print(bcolors.FAIL + "404 - Dataset  Not Found in 'deividt' Account\n" + bcolors.ENDC)
   else:
       assert 1==2,list(process_info)[-1]



def fetchModelsFromKaggle():
  ! kaggle datasets download -d deividt/snake-classifier-pretrainedmodels #Download dataset
  ! unzip \*.zip && rm *.zip #Unzip downloaded file and delete zip

def kaggle_createDataset(public=False):
  if public: 
    !kaggle datasets create -p  /content/model -u
  else: 
    !kaggle datasets create -p  /content/model

def kaggle_updateDataset(commit_message='Updating files'):
   !kaggle datasets version -p /content/model/ -m commit_message

def kaggle_setConfig(dataset_title="testing"):

  fileName="data/dataset-metadata.json"
  !kaggle datasets init -p  /content/model/
  addTitle2JSON(fileName,dataset_title)


def kaggle_saveModel():
  
  res = input("Do you want upload model  in  your kaggle Account? [y/n] ")
  
  if  res.lower()!="y" :
    return print("Upload canceled")

  res = readOption( msg = "Create new dataset for saving models[1] or update old one[2]? [1/2]")
  
  dataset_title = input("Enter dataset name: ")
  assert len(res)>0 , "Invalid length "

  kaggle_setConfig(dataset_title)

  if res=="1": #Create new dataset
    res = readOption( msg = "Create dataset Private[1] or Public [2]? [1/2]" )
    public_dataset = True if int(res)==2 else False
    kaggle_createDataset(public_dataset)
  elif res=="2": #Update dataset
    commit_msg = input ("Enter commit message (Ex. 'version 1.4') : ")
    kaggle_updateDataset(commit_msg)



True

### Model Tools

In [23]:
def loadModel(name):
  try:
    return load_model(name)
  except:
    raise Exception("Model not found")  

  

def fetchModel():
  print(bcolors.WARNING + " Downloading models from Kaggle..." + bcolors.ENDC)  

  fetchModelsFromKaggle()
  print(bcolors.OKGREEN + " Models downloaded in your local folder" + bcolors.ENDC)  
  print("---------------------------------")
  model_name = input ("\nEnter model file_name: ( Ex: model_a.h5 ) ")
  print(bcolors.WARNING + " Searching model..." + bcolors.ENDC)  

  m = loadModel(model_name)
  print(bcolors.OKGREEN + f"Model {model_name[:-3]} loaded successfully " + bcolors.ENDC)  
  return m

def getModel():

  res = readOption(" Create new model [1] or load pre-trained model from Kaggle [2]  [1/2] ")

  if res =='1':
    model_name = input(" Set  model name: (Ex. model_A) ")
    model = createModel( name= model_name ,num_breeds = 10 )
    print(bcolors.OKGREEN + f"Model {model_name[:-3]} created successfully " + bcolors.ENDC)  
  else:
    if checkIfFileExist('kaggle.json')==False:
      loadCredentials()
    model = fetchModel();
  return model

def model2TFLite(model):
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tfmodel = converter.convert()
  open(model.name+'.tflite','wb').write(tfmodel)
  print("Model successfully convert to .tflite")


### Extra Tools

In [24]:
def readOption(msg):

  res = input(msg)
  assert res=='1' or res=='2',"Invalid option"
  return res

def decode_breedID(idx):
  df_decode = pd.read_csv('decode_breed.csv')
  breed_encode = df_decode['breed_encode']
  return df_decode.loc[breed_encode == idx]['breed'].tolist()[0]

def custom_preprocess_input(x):


  if not issubclass(x.dtype.type, np.floating):
    x = x.astype('float32', copy=False)
  print(x.dtype.type)

      # 'RGB'->'BGR'
 # x = x[..., ::-1]
  mean = [103.939, 116.779, 123.68]
  # Zero-center by mean pixel
  x[..., 0] -= mean[0]
  x[..., 1] -= mean[1]
  x[..., 2] -= mean[2]

  return x

### Training Tools

In [8]:
def train(model,x_train,y_train,x_test,y_test , batch_size, epochs, model_name=""):
    model.fit(x_train, y_train, epochs=epochs,
              batch_size=batch_size,
              validation_data=(x_test, y_test))
    metrics = model.evaluate(x_test, y_test)
    return {k:v for k,v in zip (model.metrics_names, metrics)}



### File Tools

In [9]:
def addTitle2JSON(fileName,dataset_title):
    data = readFile(fileName)
    data = addTitle2Data(data,dataset_title)
    writeFile(fileName,data)
    
def writeFile(fileName,data):
  with open(fileName,'w') as json_file:
      # Guardar la informacion
      json.dump(data,json_file)
      print("File "+fileName+" updated")

def readFile(fileName):
  with open(fileName, "r") as json_file: 
     return json.load(json_file)

def addTitle2Data(data,title):
  data['id'] = data['id'].split('/')[0]+'/'+title
  data['title'] = title
  return data

def checkIfFileExist(file_name):
    return os.path.exists(os.path.abspath(file_name))

## Convolutional network arquitecture

- Using VGG16 Arquitecture

In [10]:
#Custom model
def createModel(num_breeds,name="model", ) :

  vgg= VGG16( 
      input_shape= (224,224,3), 
      weights='imagenet',
      include_top=False
      )

  #vgg.summary()
  custom_model=vgg

  for layer in custom_model.layers:
    layer.trainable = False
      
  last_layer = Flatten()(custom_model.output)
  last_layer = Dense(128, activation='relu', name='full_connected_1')(last_layer)
  last_layer = Dense(128, activation='relu', name='full_connected_2')(last_layer)
  out = Dense(num_breeds, activation='softmax', name='output')(last_layer)
  custom_model = Model(custom_model.input, out,name=name)

  
  opt = Adam(lr=0.001) #Stochastic gradient descent
  custom_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


  return custom_model

- Create a new model or load pre-trained model from Kaggle

In [25]:
model = getModel()

 Create new model [1] or load pre-trained model from Kaggle [2]  [1/2] 2
 80% 65.0M/81.0M [00:00<00:00, 71.4MB/s]
100% 81.0M/81.0M [00:00<00:00, 137MB/s] 
Archive:  snake-classifier-pretrainedmodels.zip
replace baseline_model.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
 Models downloaded in your local folder
---------------------------------

Enter model file_name: ( Ex: model_a.h5 ) baseline_model.h5
 Searching model...
Model baseline_model loaded successfully 


- Model summary

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Training and validation


- Fetch dataset from kaggle

In [31]:
fetchDatasetFromKaggle( 'snake-breeds')

Data from kaggle successfully fetch

Unzipping data... 

Archive:  snake-breeds.zip
  inflating: group_1.h5              
  inflating: group_10.h5             
  inflating: group_11.h5             
  inflating: group_12.h5             
  inflating: group_13.h5             
  inflating: group_14.h5             
  inflating: group_15.h5             
  inflating: group_16.h5             
  inflating: group_17.h5             
  inflating: group_18.h5             
  inflating: group_19.h5             
  inflating: group_2.h5              
  inflating: group_20.h5             
  inflating: group_21.h5             
  inflating: group_22.h5             
  inflating: group_23.h5             
  inflating: pre_proccessed_data.csv  
Data is ready in your local folder!



- Training and validate just certain amout of groups because of RAM limits

In this case is training just with eight groups

In [ ]:

for i in range(1,8):

    destination_filepath = BASE_PATH+'/v6/batchs/'+'group_'+str(i)+'.h5'
    with h5py.File(destination_filepath, "r") as f:
        
        print("Group:",i)
        
        x = f["input_data"][:]
        y = f["input_labels"][:]
        x=x.reshape(x.shape[0],224,224,3)
        x=preprocess_input(x*255)
        
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20)
    
    #Displaying information just for the first group
    if i ==1 :
      print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
      print("\nDistribution of train classes")
      print(pd.Series(y_train).value_counts())
      print("\nDistribution of test classes")
      print(pd.Series(y_test).value_counts())

    train(model,x_train,y_train,x_test,y_test , batch_size=32, epochs=10, model_name="modelA")

Group: 1
Epoch 1/10
50/50 [==============================] - 47s 220ms/step - loss: 7.5805 - accuracy: 0.2200 - val_loss: 3.8831 - val_accuracy: 0.2700
Epoch 2/10
50/50 [==============================] - 8s 154ms/step - loss: 1.7991 - accuracy: 0.6006 - val_loss: 3.7275 - val_accuracy: 0.3625
Epoch 3/10
50/50 [==============================] - 8s 155ms/step - loss: 0.5854 - accuracy: 0.8263 - val_loss: 3.8327 - val_accuracy: 0.3875
Epoch 4/10
50/50 [==============================] - 8s 156ms/step - loss: 0.2718 - accuracy: 0.9319 - val_loss: 4.3198 - val_accuracy: 0.3875
Epoch 5/10
50/50 [==============================] - 8s 160ms/step - loss: 0.1061 - accuracy: 0.9787 - val_loss: 4.3014 - val_accuracy: 0.3775
Epoch 6/10
50/50 [==============================] - 8s 163ms/step - loss: 0.0311 - accuracy: 0.9969 - val_loss: 4.2251 - val_accuracy: 0.3825
Epoch 7/10
50/50 [==============================] - 8s 165ms/step - loss: 0.0175 - accuracy: 0.9987 - val_loss: 4.1393 - val_accuracy: 0.3

## Testing

-Testing with images from google

In [ ]:

#Set random image
img = cv2.imread('agkistrodon-contortrix.png')

img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC) #Resize
plt.imshow(img) #Display Image
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img= img.reshape(1,224,224,3)
img = custom_preprocess_input(img)
yhat = model.predict(img)

index = yhat.argmax()
print(decode_breedID( index ))
np.max(yhat)

## Saving model

- Local save

In [ ]:
#Saving model
!mkdir model
model.save('model/'model.name+".h5")

- Saving model in Kaggle

In [ ]:
kaggle_saveModel()

Do you want upload model  in  your kaggle Account? [y/n] n
Upload canceled


#Convert model to Tensorflow Lite

In [ ]:
model2TFLite(model)

INFO:tensorflow:Assets written to: /tmp/tmpxunj5ubq/assets


INFO:tensorflow:Assets written to: /tmp/tmpxunj5ubq/assets


71785940